In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk, messagebox
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import re
from datetime import datetime
import os

DB_PATH = "delivery_service.db"
ORDERS_CSV = "Orders.csv"
RESTAURANTS_CSV = "restaurant_info.csv"

# -----------------------------
# Helpers: cleaning & utilities
# -----------------------------
EMAIL_REGEX = re.compile(r"^[A-Za-z]+@[A-Za-z]+\.[A-Za-z]+$")  # ccc@aaa.bbb (letters only)

def strip_cols(df):
    df = df.copy()
    df.columns = [c.strip().replace(" ", "_").replace("-", "_").replace("(", "").replace(")", "") for c in df.columns]
    return df

def parse_date_series(s):
    def p(x):
        if pd.isna(x): return pd.NaT
        for fmt in ("%Y-%m-%d","%d/%m/%Y","%m/%d/%Y","%d-%m-%Y","%Y/%m/%d"):
            try: return datetime.strptime(str(x), fmt)
            except: pass
        return pd.to_datetime(x, errors="coerce")
    return s.map(p)

def numify(s):
    return pd.to_numeric(pd.Series(s).astype(str).str.replace(r"[^0-9.\-]", "", regex=True), errors="coerce")

def sc(df, *names):
    for n in names:
        if n in df.columns: return n
    return None

# -----------------------------
# Build normalized database (3NF)
# -----------------------------
def build_normalized_db():
    if os.path.exists(DB_PATH):
        os.remove(DB_PATH)
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.executescript("""
    PRAGMA foreign_keys = ON;

    CREATE TABLE Restaurants(
        RestaurantID TEXT PRIMARY KEY,
        RestaurantName TEXT NOT NULL,
        Cuisine TEXT, Zone TEXT, Category TEXT, Store TEXT,
        Manager TEXT, Years_as_manager INTEGER, Email TEXT, Address TEXT
    );

    CREATE TABLE Customers(
        CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
        FirstName TEXT, LastName TEXT
    );

    CREATE TABLE PaymentModes(
        ModeID INTEGER PRIMARY KEY AUTOINCREMENT,
        ModeName TEXT UNIQUE
    );

    CREATE TABLE PaymentProviders(
        ProviderID INTEGER PRIMARY KEY AUTOINCREMENT,
        ProviderName TEXT UNIQUE
    );

    CREATE TABLE Orders(
        OrderID TEXT PRIMARY KEY,
        RestaurantID TEXT NOT NULL,
        CustomerID INTEGER,
        OrderDate TEXT,
        QuantityOfItems INTEGER,
        OrderAmount REAL,
        PaymentModeID INTEGER,
        DeliveryTimeMins REAL,
        CustomerRatingFood REAL,
        CustomerRatingDelivery REAL,
        PaymentProviderID INTEGER,
        FOREIGN KEY (RestaurantID) REFERENCES Restaurants(RestaurantID) ON UPDATE CASCADE ON DELETE RESTRICT,
        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID) ON UPDATE CASCADE ON DELETE SET NULL,
        FOREIGN KEY (PaymentModeID) REFERENCES PaymentModes(ModeID) ON UPDATE CASCADE ON DELETE SET NULL,
        FOREIGN KEY (PaymentProviderID) REFERENCES PaymentProviders(ProviderID) ON UPDATE CASCADE ON DELETE SET NULL
    );

    -- Extension required by brief (no need to populate for the GUI):
    CREATE TABLE IF NOT EXISTS DeliveryStaff(
        StaffID INTEGER PRIMARY KEY AUTOINCREMENT,
        StaffName TEXT NOT NULL,
        VehicleType TEXT CHECK (VehicleType IN ('car','motorbike','bike'))
    );
    CREATE TABLE IF NOT EXISTS Deliveries(
        DeliveryID INTEGER PRIMARY KEY AUTOINCREMENT,
        OrderID TEXT NOT NULL UNIQUE,
        StaffID INTEGER NOT NULL,
        FOREIGN KEY (OrderID) REFERENCES Orders(OrderID) ON UPDATE CASCADE ON DELETE CASCADE,
        FOREIGN KEY (StaffID) REFERENCES DeliveryStaff(StaffID) ON UPDATE CASCADE ON DELETE RESTRICT
    );
    """)

    # Read & clean CSVs
    r_df_raw = pd.read_csv(RESTAURANTS_CSV)
    o_df_raw = pd.read_csv(ORDERS_CSV)

    r = strip_cols(r_df_raw)
    o = strip_cols(o_df_raw)

    # Orders: align expected names
    # Expected: Order_ID, First_Customer_Name, Last_Customer_Name, Restaurant_ID, Order_Date,
    # Quantity_of_Items, Order_Amount, Payment_Mode, Delivery_Time_Taken_mins, Customer_Rating_Food, Customer_Rating_Delivery, Card_provider
    #
    qty = sc(o, "Quantity_of_Items")
    amt = sc(o, "Order_Amount")
    dmins = sc(o, "Delivery_Time_Taken_mins", "Delivery_Time_Taken", "Delivery_Time_Taken__mins")
    fr = sc(o, "Customer_Rating_Food")
    dr = sc(o, "Customer_Rating_Delivery")
    odt = sc(o, "Order_Date")
    oid = sc(o, "Order_ID", "OrderId", "OrderID")
    rid = sc(o, "Restaurant_ID", "RestaurantID")
    first = sc(o, "First_Customer_Name", "FirstName")
    last = sc(o, "Last_Customer_Name", "LastName")
    mode = sc(o, "Payment_Mode")
    prov = sc(o, "Card_provider", "Card_Provider")

    # Coerce
    if odt: o[odt] = parse_date_series(o[odt])
    for c in [qty]: 
        if c: o[c] = pd.to_numeric(o[c], errors="coerce").astype("Int64")
    for c in [amt, dmins, fr, dr]:
        if c:
            o[c] = numify(o[c])
    # Restaurants
    r_id = sc(r, "RestaurantID", "Restaurant_Id")
    r_name = sc(r, "RestaurantName", "Restaurant_Name")
    years = sc(r, "Years_as_manager","YearsAsManager")
    email = sc(r, "Email","email")

    if years: r[years] = pd.to_numeric(r[years], errors="coerce").fillna(0).astype(int)

    # Insert Restaurants
    for _, row in r.iterrows():
        cur.execute("""
            INSERT OR REPLACE INTO Restaurants
            (RestaurantID, RestaurantName, Cuisine, Zone, Category, Store, Manager, Years_as_manager, Email, Address)
            VALUES (?,?,?,?,?,?,?,?,?,?)
        """, (
            row.get(r_id), row.get(r_name),
            row.get("Cuisine"), row.get("Zone"), row.get("Category"), row.get("Store"),
            row.get("Manager"), row.get(years), row.get(email), row.get("Address")
        ))

    # lookup values
    if mode:
        for v in pd.Series(o[mode]).dropna().astype(str).str.strip().unique():
            if v: cur.execute("INSERT OR IGNORE INTO PaymentModes (ModeName) VALUES(?);", (v,))
    if prov:
        for v in pd.Series(o[prov]).dropna().astype(str).str.strip().unique():
            if v: cur.execute("INSERT OR IGNORE INTO PaymentProviders (ProviderName) VALUES(?);", (v,))

    # Build lookup maps
    mode_map = {name: mid for (mid, name) in cur.execute("SELECT ModeID, ModeName FROM PaymentModes;").fetchall()}
    prov_map = {name: pid for (pid, name) in cur.execute("SELECT ProviderID, ProviderName FROM PaymentProviders;").fetchall()}

    # Insert Customers (simple name-based dedupe)
    cust_map = {}
    if first and last:
        for _, row in o[[first,last]].dropna().drop_duplicates().iterrows():
            k = (str(row[first]).strip(), str(row[last]).strip())
            cur.execute("INSERT INTO Customers(FirstName, LastName) VALUES(?,?)", k)
            cust_map[k] = cur.lastrowid

    # Insert Orders
    for _, row in o.iterrows():
        k = None
        if first and last:
            k = (str(row.get(first) or "").strip(), str(row.get(last) or "").strip())
        cust_id = cust_map.get(k)

        # date as ISO string
        dt = row.get(odt)
        if isinstance(dt, pd.Timestamp): dt = dt.strftime("%Y-%m-%d")
        elif isinstance(dt, datetime): dt = dt.strftime("%Y-%m-%d")
        else: dt = None

        mode_id = None
        if mode and isinstance(row.get(mode), str):
            mode_id = mode_map.get(row.get(mode).strip())
        prov_id = None
        if prov and isinstance(row.get(prov), str):
            prov_id = prov_map.get(row.get(prov).strip())

        if row.get(oid) and row.get(rid):
            cur.execute("""
                INSERT OR REPLACE INTO Orders
                (OrderID, RestaurantID, CustomerID, OrderDate, QuantityOfItems, OrderAmount,
                 PaymentModeID, DeliveryTimeMins, CustomerRatingFood, CustomerRatingDelivery, PaymentProviderID)
                VALUES (?,?,?,?,?,?,?,?,?,?,?)
            """, (
                str(row.get(oid)), str(row.get(rid)), cust_id, dt,
                None if qty is None or pd.isna(row.get(qty)) else int(row.get(qty)),
                None if amt is None or pd.isna(row.get(amt)) else float(row.get(amt)),
                mode_id,
                None if dmins is None or pd.isna(row.get(dmins)) else float(row.get(dmins)),
                None if fr is None or pd.isna(row.get(fr)) else float(row.get(fr)),
                None if dr is None or pd.isna(row.get(dr)) else float(row.get(dr)),
                prov_id
            ))

    conn.commit()
    conn.close()

# -----------------------------
# GUI (Tkinter, spec-compliant)
# -----------------------------
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Delivery Service – Manager & Analysis")
        self.geometry("1200x800")

        self.style = ttk.Style()
        self.style.configure("TNotebook", tabposition="nw")
        self.style.configure("TButton", padding=(8,4), width=22)

        nb = ttk.Notebook(self)
        nb.pack(expand=True, fill="both")

        self.tab_manager = ttk.Frame(nb)
        self.tab_rest = ttk.Frame(nb)
        self.tab_orders = ttk.Frame(nb)
        self.tab_calc = ttk.Frame(nb)

        nb.add(self.tab_manager, text="Manager Update")
        nb.add(self.tab_rest, text="Restaurants")
        nb.add(self.tab_orders, text="Orders")
        nb.add(self.tab_calc, text="Calculations")

        self._build_manager_tab()
        self._build_rest_tab()
        self._build_orders_tab()
        self._build_calc_tab()

    # ---- Manager Update (email ccc@aaa.bbb) ----
    def _build_manager_tab(self):
        lf = ttk.LabelFrame(self.tab_manager, text="Update Manager (email must be letters-only: ccc@aaa.bbb)")
        lf.pack(padx=12, pady=12, fill="x")

        ttk.Label(lf, text="RestaurantID").grid(row=0, column=0, padx=8, pady=8, sticky="e")
        ttk.Label(lf, text="Manager").grid(row=1, column=0, padx=8, pady=8, sticky="e")
        ttk.Label(lf, text="Email").grid(row=2, column=0, padx=8, pady=8, sticky="e")
        ttk.Label(lf, text="Years as manager").grid(row=3, column=0, padx=8, pady=8, sticky="e")

        self.e_rid = ttk.Entry(lf, width=28)
        self.e_mgr = ttk.Entry(lf, width=28)
        self.e_email = ttk.Entry(lf, width=28)
        self.e_years = ttk.Entry(lf, width=28)
        self.e_rid.grid(row=0, column=1, padx=8, pady=8)
        self.e_mgr.grid(row=1, column=1, padx=8, pady=8)
        self.e_email.grid(row=2, column=1, padx=8, pady=8)
        self.e_years.grid(row=3, column=1, padx=8, pady=8)

        ttk.Button(lf, text="Update", command=self.update_manager).grid(row=4, column=0, columnspan=2, pady=12)

    def update_manager(self):
        rid = self.e_rid.get().strip()
        mgr = self.e_mgr.get().strip() or None
        email = self.e_email.get().strip() or None
        years = self.e_years.get().strip() or None

        if not rid:
            messagebox.showerror("Error", "RestaurantID is required.")
            return
        if email and not EMAIL_REGEX.match(email):
            messagebox.showerror("Error", "Email must match ccc@aaa.bbb (letters only).")
            return
        years_val = None
        if years:
            try: years_val = int(years)
            except: 
                messagebox.showerror("Error", "Years as manager must be an integer.")
                return

        conn = sqlite3.connect(DB_PATH)
        cur = conn.cursor()
        try:
            cur.execute("SELECT 1 FROM Restaurants WHERE RestaurantID=?;", (rid,))
            if not cur.fetchone():
                messagebox.showerror("Error", f"RestaurantID '{rid}' not found.")
                return
            cur.execute("""
                UPDATE Restaurants
                SET Manager=COALESCE(?, Manager),
                    Email=COALESCE(?, Email),
                    Years_as_manager=COALESCE(?, Years_as_manager)
                WHERE RestaurantID=?;
            """, (mgr, email, years_val, rid))
            conn.commit()
        finally:
            conn.close()
        messagebox.showinfo("Success", f"Updated manager info for RestaurantID={rid}")

    # ---- Restaurants tab ----
    def _build_rest_tab(self):
        btn = ttk.Button(self.tab_rest, text="Show Restaurants", command=self.show_restaurants)
        btn.pack(pady=10)
        self.txt_rest = tk.Text(self.tab_rest, height=28, width=160)
        self.txt_rest.pack(padx=10, pady=10)

    def show_restaurants(self):
        conn = sqlite3.connect(DB_PATH)
        try:
            df = pd.read_sql_query("SELECT * FROM Restaurants ORDER BY RestaurantName;", conn)
        finally:
            conn.close()
        self.txt_rest.config(state=tk.NORMAL)
        self.txt_rest.delete("1.0", tk.END)
        self.txt_rest.insert(tk.END, df.to_string(index=False))
        self.txt_rest.config(state=tk.DISABLED)

    # ---- Orders tab ----
    def _build_orders_tab(self):
        btn = ttk.Button(self.tab_orders, text="Show Sample of Orders", command=self.show_orders)
        btn.pack(pady=10)
        self.txt_orders = tk.Text(self.tab_orders, height=28, width=160)
        self.txt_orders.pack(padx=10, pady=10)

    def show_orders(self):
        conn = sqlite3.connect(DB_PATH)
        try:
            df = pd.read_sql_query("""
                SELECT OrderID, RestaurantID, CustomerID, OrderDate, QuantityOfItems, OrderAmount,
                       DeliveryTimeMins, CustomerRatingFood, CustomerRatingDelivery
                FROM Orders
                ORDER BY OrderDate LIMIT 200;
            """, conn)
        finally:
            conn.close()
        self.txt_orders.config(state=tk.NORMAL)
        self.txt_orders.delete("1.0", tk.END)
        self.txt_orders.insert(tk.END, df.to_string(index=False))
        self.txt_orders.config(state=tk.DISABLED)

    # ---- Calculations tab ----
    def _build_calc_tab(self):
        bar = ttk.Frame(self.tab_calc)
        bar.pack(pady=10)
        ttk.Button(bar, text="Draw Delivery Time Histogram", command=self.draw_histogram).pack(side="left", padx=6)
        ttk.Button(bar, text="Mean Food Rating per Restaurant", command=self.show_mean_ratings).pack(side="left", padx=6)

        self.tree = ttk.Treeview(self.tab_calc, columns=("RestaurantID","RestaurantName","MeanFood","NumOrders"), show="headings", height=18)
        for c, w in [("RestaurantID",140), ("RestaurantName",300), ("MeanFood",160), ("NumOrders",120)]:
            self.tree.heading(c, text=c); self.tree.column(c, width=w, anchor="center")
        self.tree.pack(expand=True, fill="both", padx=10, pady=10)

        # Canvas holder for histogram
        self.hist_frame = ttk.LabelFrame(self.tab_calc, text="Histogram")
        self.hist_frame.pack(fill="both", padx=10, pady=10)
        self.canvas_widget = None

    def draw_histogram(self):
        conn = sqlite3.connect(DB_PATH)
        try:
            df = pd.read_sql_query("SELECT DeliveryTimeMins FROM Orders WHERE DeliveryTimeMins IS NOT NULL;", conn)
        finally:
            conn.close()
        data = df["DeliveryTimeMins"].dropna().astype(float)
        if data.empty:
            messagebox.showinfo("Info", "No delivery time data available.")
            return

        fig, ax = plt.subplots()
        ax.hist(data, bins=20, edgecolor="black")
        ax.set_xlabel("Delivery Time (mins)"); ax.set_ylabel("Frequency")
        ax.set_title("Delivery Time Histogram")
        if self.canvas_widget:
            self.canvas_widget.get_tk_widget().destroy()
        canvas = FigureCanvasTkAgg(fig, master=self.hist_frame)
        canvas.draw()
        self.canvas_widget = canvas
        canvas.get_tk_widget().pack()
        # No second mainloop!

    def show_mean_ratings(self):
        conn = sqlite3.connect(DB_PATH)
        try:
            df = pd.read_sql_query("""
                SELECT o.RestaurantID, r.RestaurantName,
                       AVG(o.CustomerRatingFood) AS MeanCustomerRatingFood,
                       COUNT(*) AS NumOrders
                FROM Orders o
                LEFT JOIN Restaurants r ON r.RestaurantID = o.RestaurantID
                GROUP BY o.RestaurantID, r.RestaurantName
                ORDER BY MeanCustomerRatingFood DESC;
            """, conn)
        finally:
            conn.close()
        for row in self.tree.get_children():
            self.tree.delete(row)
        for _, row in df.iterrows():
            self.tree.insert("", "end", values=(
                row["RestaurantID"],
                row["RestaurantName"],
                "" if pd.isna(row["MeanCustomerRatingFood"]) else round(float(row["MeanCustomerRatingFood"]), 3),
                int(row["NumOrders"])
            ))

# -----------------------------
# Main
# -----------------------------
if __name__ == "__main__":
    # Build normalized DB from CSVs (idempotent for your current folder)
    build_normalized_db()
    # Launch GUI
    App().mainloop()

    
    